In [1]:
%%html
<style>@import url('../style.css')</style><script>IPython.OutputArea.prototype._should_scroll = function(){return false}</script>

In [ ]:
try:
    IMPORT_PATH
except NameError:
    IMPORT_PATH = ''
    
%run -i {IMPORT_PATH + 'Dataset.ipynb'}

import theano
import pylearn2 as pl2
import pylearn2.models.mlp
import pylearn2.costs.mlp.dropout
import pylearn2.training_algorithms.sgd
import pylearn2.termination_criteria
import pylearn2.space
from pylearn2.models.mlp import ConvRectifiedLinear as ConvReLU
from pylearn2.models.mlp import RectifiedLinear as ReLU
from pylearn2.models.mlp import Softmax

def learn_network(plot_filename='', class_count=10, epochs=100, learning_rate=0.002, filter_count=80,
                  dropout=[1.0, 0.5, 1.0, 0.5, 0.5], batch_size=1000):

    cost = pl2.costs.cost.SumOfCosts([pl2.costs.mlp.dropout.Dropout(input_include_probs={
                'c0': dropout[0],
                'c1': dropout[1],
                'h0': dropout[2],
                'h1': dropout[3],
                'out': dropout[4]
    }),
    pl2.costs.mlp.WeightDecay({
                'c0': 0.001,
                'c1': 0.001,
                'h0': 0.001,
                'h1': 0.001,
                'out': 0.001
    }) ])
    
    layers = [
        ConvReLU(layer_name='c0', kernel_shape=[57, 6], pool_shape=[4, 3], pool_stride=[1, 3], pool_type='max',
                 output_channels=filter_count, irange=0.001, init_bias=0.1),
        ConvReLU(layer_name='c1', kernel_shape=[1, 3], pool_shape=[1, 3], pool_stride=[1, 3], pool_type='max',
                 output_channels=filter_count, irange=0.1),
        ReLU(layer_name='h0', dim=5000, istdev=0.01),
        ReLU(layer_name='h1', dim=5000, istdev=0.01),
        Softmax(layer_name='out', n_classes=class_count, istdev=0.01)
    ]
    
    if NET_WINDOW_SIZE == 'short':
        model = pl2.models.mlp.MLP(layers, batch_size=batch_size, input_space=pl2.space.Conv2DSpace([60, 41], channels=2))
    else:
        model = pl2.models.mlp.MLP(layers, batch_size=batch_size, input_space=pl2.space.Conv2DSpace([60, 101], channels=2))
        
    trainer = pl2.training_algorithms.sgd.SGD(learning_rate=learning_rate, cost=cost,
               learning_rule=pl2.training_algorithms.learning_rule.Momentum(init_momentum=0.9, nesterov_momentum=True),
               monitoring_dataset={'training': dataset.train, 'validation': dataset.validation},
               termination_criterion=pl2.termination_criteria.EpochCounter(epochs - 1),                                                  
               train_iteration_mode='even_shuffled_sequential', monitor_iteration_mode='even_shuffled_sequential')
    
    training_misclass = []
    validation_misclass = []
    
    try:
        gc.collect()
        trainer.setup(model, dataset.train)
        f = plt.figure(figsize=(12, 12))

        while True:
            start_time = time.time()
            logging.disable(logging.INFO)
            
            trainer.train(dataset=dataset.train)
            model.monitor.report_epoch()
            model.monitor()
            training_misclass.append(pl2.monitor.read_channel(model, 'training_out_misclass'))
            validation_misclass.append(1 - dataset.validate_model(model, False))
            
            time_elapsed = time.time() - start_time
            logging.disable(logging.NOTSET)            

            # Plot
            cmap = sb.diverging_palette(220, 10, as_cmap=True)
            cmap_delta = sb.diverging_palette(220, 19, as_cmap=True)
            
            ax = plt.subplot2grid((10, filter_count // 4), (0, 0), colspan=filter_count // 4, rowspan=2)
            x = range(len(training_misclass))
            ax.plot(x, training_misclass, 'g-', validation_misclass, 'r-')
            ax.set_ylim((0.0, 1.0))
            ax.set_xlim((0.0, epochs))
            ax.get_xaxis().set_visible(False)
            plt.title('Training accuracy: {}% / Validation accuracy: {}% | Epoch {} | Loss {} | Time/epoch: {} s'.format(
                    to_percentage(1 - training_misclass[-1]), to_percentage(1 - validation_misclass[-1]),
                    model.monitor.get_epochs_seen(), str(pl2.monitor.read_channel(model, 'training_objective'))[0:5],
                    int(time_elapsed)))

            for i in range(filter_count):
                ax = plt.subplot2grid((10, filter_count // 4), (i / (filter_count // 4) * 2 + 2, i % (filter_count // 4)))
                ax.imshow(model.layers[0].get_weights_topo()[i, :, :, 0], origin='lower', aspect='auto', cmap=cmap, interpolation='nearest')
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)
                ax = plt.subplot2grid((10, filter_count // 4), (i / (filter_count // 4) * 2 + 3, i % (filter_count // 4)))
                ax.imshow(model.layers[0].get_weights_topo()[i, :, :, 1], origin='lower', aspect='auto', cmap=cmap_delta, interpolation='nearest')
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)

            f.subplots_adjust(wspace=0.0)
            IPython.display.display(plt.gcf())
            IPython.display.clear_output(wait=True)

            plt.savefig(plot_filename + '.png', bbox_inches='tight')
            plt.savefig(plot_filename + '.pdf', bbox_inches='tight')

            if not trainer.continue_learning(model):
                return model
    except KeyboardInterrupt:
        return model

In [3]:
%run -i {IMPORT_PATH + 'Evaluation.ipynb'}